In [ ]:
!unzip extracted_frames.zip


Archive:  extracted_frames.zip
replace extracted_frames/frame1000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import glob

In [ ]:
def find_scale_universal(image, scale_length=1000):

    height, width = image.shape[:2]

    scale_region = image[height-30:height, :]

    gray = cv2.cvtColor(scale_region, cv2.COLOR_BGR2GRAY)

    _, binary = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        widest_contour = None
        max_width = 0

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)

            if w > max_width and w > 100 and h < 20:
                max_width = w
                widest_contour = contour

        if widest_contour is not None:
            x, y, w, h = cv2.boundingRect(widest_contour)
            print(f"The scale was found: {w} px = {scale_length} µm")

            calibration_factor = w / scale_length
            return calibration_factor

    print("The scale was not found")
    return None

In [ ]:
def process_bubble_image_universal(image, calibration_factor):

    height, width = image.shape[:2]

    bubble_region = image[0:height-100, :]

    gray = cv2.cvtColor(bubble_region, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)

    blurred = cv2.GaussianBlur(enhanced, (9, 9), 0)

    _, binary = cv2.threshold(blurred, 80, 255, cv2.THRESH_BINARY_INV)

    kernel = np.ones((5, 5), np.uint8)
    binary_cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=3)
    binary_cleaned = cv2.morphologyEx(binary_cleaned, cv2.MORPH_OPEN, kernel, iterations=1)

    contours, _ = cv2.findContours(binary_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None

    main_contour = max(contours, key=cv2.contourArea)

    area_px = cv2.contourArea(main_contour)
    perimeter_px = cv2.arcLength(main_contour, True)
    equivalent_radius_px = np.sqrt(area_px / np.pi)

    if area_px < 100:
        return None

    area_um = area_px / (calibration_factor ** 2)
    perimeter_um = perimeter_px / calibration_factor
    radius_um = equivalent_radius_px / calibration_factor



    return {
        'area_um': area_um,
        'perimeter_um': perimeter_um,
        'radius_um': radius_um
    }

In [ ]:
def find_calibration_for_concentration(image_paths, concentration):

    scale_lengths = {
        0: 1000, 5: 500, 12.5: 500, 25: 1000,
        50: 1000, 75: 500, 96: 500
    }
    scale_length = scale_lengths.get(concentration, 1000)


    for i, image_path in enumerate(image_paths):
        print(f"  Try {i+1}/{len(image_paths)}: {os.path.basename(image_path)}")

        image = cv2.imread(image_path)
        if image is not None:
            calibration = find_scale_universal(image, scale_length)
            if calibration:
                print(f"Scale {os.path.basename(image_path)}")
                return calibration

    print("We have not scale")
    return None

In [ ]:
def add_concentration_data(image_paths, concentration, output_csv='bubble_analysis_um.csv'):


    if not os.path.exists(output_csv):
        return None

    existing_df = pd.read_csv(output_csv)
    print(f"The table was done, lenght: {len(existing_df)}")

    calibration_factor = find_calibration_for_concentration(image_paths, concentration)

    if not calibration_factor:
        print(f"No colibration {concentration}%!")
        return None

    results = []

    for i, image_path in enumerate(image_paths):
        print(f"Working {i+1}/{len(image_paths)}: {os.path.basename(image_path)}")

        image = cv2.imread(image_path)
        if image is None:
            continue

        filename = os.path.basename(image_path)
        frame_number = ''.join(filter(str.isdigit, filename)) or i + 1
        frame_number = int(frame_number)

        bubble_data = process_bubble_image_universal(image, calibration_factor)

        if bubble_data is not None:
            results.append({
                'Image number': frame_number,
                'Square': round(bubble_data['area_um'], 2),
                'Radius': round(bubble_data['radius_um'], 2),
                'Circumference': round(bubble_data['perimeter_um'], 2),
                'Alcohol concentration': concentration
            })


    if results:
        new_df = pd.DataFrame(results)
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
        combined_df.to_csv(output_csv, index=False)



        return combined_df
    else:
        print("No frames")
        return existing_df

In [ ]:
def create_empty_table(output_csv='bubble_analysis_um.csv'):

    empty_df = pd.DataFrame(columns=[
        'Image number',
        'Square',
        'Radius',
        'Circumference',
        'Alcohol concentration'
    ])

    empty_df.to_csv(output_csv, index=False, encoding='utf-8')
    print(f"Emty table: {output_csv}")

In [ ]:
create_empty_table("bubble_analysis_um.csv")

Emty table: bubble_analysis_um.csv


In [ ]:
image_paths_0 = glob.glob("/content/extracted_frames/*.jpg")
image_paths_0.sort()
add_concentration_data(image_paths_0, 0, "bubble_analysis_um.csv")

The table was done, lenght: 0
  Try 1/588: frame1000.jpg
The scale was found: 211 px = 1000 µm
Scale frame1000.jpg
Working 1/588: frame1000.jpg
Working 2/588: frame1001.jpg
Working 3/588: frame1002.jpg
Working 4/588: frame1003.jpg
Working 5/588: frame1014.jpg
Working 6/588: frame1015.jpg
Working 7/588: frame1016.jpg
Working 8/588: frame1017.jpg
Working 9/588: frame1028.jpg
Working 10/588: frame1029.jpg
Working 11/588: frame1148.jpg
Working 12/588: frame1160.jpg
Working 13/588: frame1162.jpg
Working 14/588: frame1163.jpg
Working 15/588: frame1174.jpg
Working 16/588: frame1176.jpg
Working 17/588: frame1177.jpg
Working 18/588: frame1188.jpg
Working 19/588: frame1189.jpg
Working 20/588: frame1200.jpg
Working 21/588: frame1201.jpg
Working 22/588: frame1202.jpg
Working 23/588: frame1203.jpg
Working 24/588: frame1214.jpg
Working 25/588: frame1215.jpg
Working 26/588: frame1216.jpg
Working 27/588: frame1217.jpg
Working 28/588: frame1228.jpg
Working 29/588: frame1229.jpg
Working 30/588: frame134

/tmp/ipython-input-816284027.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([existing_df, new_df], ignore_index=True)


,Image number,Square,Radius,Circumference,Alcohol concentration
0,1000,2100278.52,817.64,5648.95,0
1,1001,2100244.83,817.64,5646.17,0
2,1002,2100626.67,817.71,5646.17,0
3,1003,2152624.60,827.77,5563.44,0
4,1014,2068900.07,811.51,5602.51,0
...,...,...,...,...,...
583,966,1975921.48,793.07,5507.24,0
584,970,2009377.60,799.75,5731.56,0
585,971,2008625.14,799.60,5734.34,0
586,972,2009096.83,799.70,5734.34,0


In [ ]:
image_paths_5 = glob.glob("/content/extracted_frames_5%/*.jpg")
image_paths_5.sort()
add_concentration_data(image_paths_5, 5, "bubble_analysis_um.csv")

Выходные данные были обрезаны до нескольких последних строк (5000).
Working 506/5505: frame1452.jpg
Working 507/5505: frame1453.jpg
Working 508/5505: frame1454.jpg
Working 509/5505: frame1455.jpg
Working 510/5505: frame1456.jpg
Working 511/5505: frame1457.jpg
Working 512/5505: frame1458.jpg
Working 513/5505: frame1459.jpg
Working 514/5505: frame146.jpg
Working 515/5505: frame1460.jpg
Working 516/5505: frame1461.jpg
Working 517/5505: frame1462.jpg
Working 518/5505: frame1463.jpg
Working 519/5505: frame1464.jpg
Working 520/5505: frame1465.jpg
Working 521/5505: frame1466.jpg
Working 522/5505: frame1467.jpg
Working 523/5505: frame1468.jpg
Working 524/5505: frame1469.jpg
Working 525/5505: frame147.jpg
Working 526/5505: frame1470.jpg
Working 527/5505: frame1471.jpg
Working 528/5505: frame1472.jpg
Working 529/5505: frame1473.jpg
Working 530/5505: frame1474.jpg
Working 531/5505: frame1475.jpg
Working 532/5505: frame1476.jpg
Working 533/5505: frame1477.jpg
Working 534/5505: frame1478.jpg
Workin

,Image number,Square,Radius,Circumference,Alcohol concentration
0,1000,2100278.52,817.64,5648.95,0.0
1,1001,2100244.83,817.64,5646.17,0.0
2,1002,2100626.67,817.71,5646.17,0.0
3,1003,2152624.60,827.77,5563.44,0.0
4,1014,2068900.07,811.51,5602.51,0.0
...,...,...,...,...,...
11593,995,301002.02,309.54,2171.87,5.0
11594,996,301218.28,309.65,2171.87,5.0
11595,997,301170.22,309.62,2171.87,5.0
11596,998,287233.28,302.37,2644.14,5.0


In [ ]:
image_paths_12_5 = glob.glob("/content/extracted_frames_12,5%/*.jpg")
image_paths_12_5.sort()
add_concentration_data(image_paths_12_5, 12.5, "bubble_analysis_um.csv")

Выходные данные были обрезаны до нескольких последних строк (5000).
Working 506/5505: frame1452.jpg
Working 507/5505: frame1453.jpg
Working 508/5505: frame1454.jpg
Working 509/5505: frame1455.jpg
Working 510/5505: frame1456.jpg
Working 511/5505: frame1457.jpg
Working 512/5505: frame1458.jpg
Working 513/5505: frame1459.jpg
Working 514/5505: frame146.jpg
Working 515/5505: frame1460.jpg
Working 516/5505: frame1461.jpg
Working 517/5505: frame1462.jpg
Working 518/5505: frame1463.jpg
Working 519/5505: frame1464.jpg
Working 520/5505: frame1465.jpg
Working 521/5505: frame1466.jpg
Working 522/5505: frame1467.jpg
Working 523/5505: frame1468.jpg
Working 524/5505: frame1469.jpg
Working 525/5505: frame147.jpg
Working 526/5505: frame1470.jpg
Working 527/5505: frame1471.jpg
Working 528/5505: frame1472.jpg
Working 529/5505: frame1473.jpg
Working 530/5505: frame1474.jpg
Working 531/5505: frame1475.jpg
Working 532/5505: frame1476.jpg
Working 533/5505: frame1477.jpg
Working 534/5505: frame1478.jpg
Workin

,Image number,Square,Radius,Circumference,Alcohol concentration
0,1000,2100278.52,817.64,5648.95,0.0
1,1001,2100244.83,817.64,5646.17,0.0
2,1002,2100626.67,817.71,5646.17,0.0
3,1003,2152624.60,827.77,5563.44,0.0
4,1014,2068900.07,811.51,5602.51,0.0
...,...,...,...,...,...
6088,995,1672092.00,729.55,7766.08,12.5
6089,996,1672502.31,729.64,7768.77,12.5
6090,997,1672502.31,729.64,7768.77,12.5
6091,998,821921.56,511.49,3849.32,12.5


In [ ]:
image_paths_12_5 = glob.glob("/content/extracted_frames_12,5%/*.jpg")
image_paths_12_5.sort()
add_concentration_data(image_paths_12_5, 12.5, "bubble_analysis_um.csv")

Выходные данные были обрезаны до нескольких последних строк (5000).
Working 506/5505: frame1452.jpg
Working 507/5505: frame1453.jpg
Working 508/5505: frame1454.jpg
Working 509/5505: frame1455.jpg
Working 510/5505: frame1456.jpg
Working 511/5505: frame1457.jpg
Working 512/5505: frame1458.jpg
Working 513/5505: frame1459.jpg
Working 514/5505: frame146.jpg
Working 515/5505: frame1460.jpg
Working 516/5505: frame1461.jpg
Working 517/5505: frame1462.jpg
Working 518/5505: frame1463.jpg
Working 519/5505: frame1464.jpg
Working 520/5505: frame1465.jpg
Working 521/5505: frame1466.jpg
Working 522/5505: frame1467.jpg
Working 523/5505: frame1468.jpg
Working 524/5505: frame1469.jpg
Working 525/5505: frame147.jpg
Working 526/5505: frame1470.jpg
Working 527/5505: frame1471.jpg
Working 528/5505: frame1472.jpg
Working 529/5505: frame1473.jpg
Working 530/5505: frame1474.jpg
Working 531/5505: frame1475.jpg
Working 532/5505: frame1476.jpg
Working 533/5505: frame1477.jpg
Working 534/5505: frame1478.jpg
Workin

,Image number,Square,Radius,Circumference,Alcohol concentration
0,1000,2100278.52,817.64,5648.95,0.0
1,1001,2100244.83,817.64,5646.17,0.0
2,1002,2100626.67,817.71,5646.17,0.0
3,1003,2152624.60,827.77,5563.44,0.0
4,1014,2068900.07,811.51,5602.51,0.0
...,...,...,...,...,...
29477,995,1672092.00,729.55,7766.08,12.5
29478,996,1672502.31,729.64,7768.77,12.5
29479,997,1672502.31,729.64,7768.77,12.5
29480,998,821921.56,511.49,3849.32,12.5


In [ ]:
image_paths_25 = glob.glob("/content/extracted_frames_25%/*.jpg")
image_paths_25.sort()
add_concentration_data(image_paths_25, 25, "bubble_analysis_um.csv")


The table was done, lenght: 20309
  Try 1/3668: frame0.jpg
The scale was found: 207 px = 1000 µm
Scale frame0.jpg
Working 1/3668: frame0.jpg
Working 2/3668: frame1.jpg
Working 3/3668: frame10.jpg
Working 4/3668: frame100.jpg
Working 5/3668: frame1000.jpg
Working 6/3668: frame1001.jpg
Working 7/3668: frame1002.jpg
Working 8/3668: frame1003.jpg
Working 9/3668: frame1004.jpg
Working 10/3668: frame1005.jpg
Working 11/3668: frame1006.jpg
Working 12/3668: frame1007.jpg
Working 13/3668: frame1008.jpg
Working 14/3668: frame1009.jpg
Working 15/3668: frame101.jpg
Working 16/3668: frame1010.jpg
Working 17/3668: frame1011.jpg
Working 18/3668: frame1012.jpg
Working 19/3668: frame1013.jpg
Working 20/3668: frame1014.jpg
Working 21/3668: frame1015.jpg
Working 22/3668: frame1016.jpg
Working 23/3668: frame1017.jpg
Working 24/3668: frame1018.jpg
Working 25/3668: frame1019.jpg
Working 26/3668: frame102.jpg
Working 27/3668: frame1020.jpg
Working 28/3668: frame1021.jpg
Working 29/3668: frame1022.jpg
Working

,Image number,Square,Radius,Circumference,Alcohol concentration
0,1000,2100278.52,817.64,5648.95,0.0
1,1001,2100244.83,817.64,5646.17,0.0
2,1002,2100626.67,817.71,5646.17,0.0
3,1003,2152624.60,827.77,5563.44,0.0
4,1014,2068900.07,811.51,5602.51,0.0
...,...,...,...,...,...
23972,995,33886.44,103.86,735.39,25.0
23973,996,42416.39,116.20,854.16,25.0
23974,997,42626.43,116.48,858.16,25.0
23975,998,62230.16,140.74,1067.41,25.0


In [ ]:
image_paths_50 = glob.glob("/content/extracted_frames_50%/*.jpg")
image_paths_50.sort()
add_concentration_data(image_paths_50, 50, "bubble_analysis_um.csv")

The table was done, lenght: 18886
  Try 1/1423: frame0.jpg
The scale was found: 211 px = 1000 µm
Scale frame0.jpg
Working 1/1423: frame0.jpg
Working 2/1423: frame1.jpg
Working 3/1423: frame10.jpg
Working 4/1423: frame100.jpg
Working 5/1423: frame1000.jpg
Working 6/1423: frame1001.jpg
Working 7/1423: frame1002.jpg
Working 8/1423: frame1003.jpg
Working 9/1423: frame1004.jpg
Working 10/1423: frame1005.jpg
Working 11/1423: frame1006.jpg
Working 12/1423: frame1007.jpg
Working 13/1423: frame1008.jpg
Working 14/1423: frame1009.jpg
Working 15/1423: frame101.jpg
Working 16/1423: frame1010.jpg
Working 17/1423: frame1011.jpg
Working 18/1423: frame1012.jpg
Working 19/1423: frame1013.jpg
Working 20/1423: frame1014.jpg
Working 21/1423: frame1015.jpg
Working 22/1423: frame1016.jpg
Working 23/1423: frame1017.jpg
Working 24/1423: frame1018.jpg
Working 25/1423: frame1019.jpg
Working 26/1423: frame102.jpg
Working 27/1423: frame1020.jpg
Working 28/1423: frame1021.jpg
Working 29/1423: frame1022.jpg
Working

,Image number,Square,Radius,Circumference,Alcohol concentration
0,1000,2100278.52,817.64,5648.95,0.0
1,1001,2100244.83,817.64,5646.17,0.0
2,1002,2100626.67,817.71,5646.17,0.0
3,1003,2152624.60,827.77,5563.44,0.0
4,1014,2068900.07,811.51,5602.51,0.0
...,...,...,...,...,...
20304,995,109487.66,186.68,2034.86,50.0
20305,996,43956.78,118.29,1821.25,50.0
20306,997,44125.24,118.51,1820.10,50.0
20307,998,18350.89,76.43,540.96,50.0


In [ ]:
image_paths_75 = glob.glob("/content/extracted_frames_75%/*.jpg")
image_paths_75.sort()
add_concentration_data(image_paths_75, 75, "bubble_analysis_um.csv")

The table was done, lenght: 16661
  Try 1/2225: frame0.jpg
The scale was found: 134 px = 500 µm
Scale frame0.jpg
Working 1/2225: frame0.jpg
Working 2/2225: frame1.jpg
Working 3/2225: frame10.jpg
Working 4/2225: frame100.jpg
Working 5/2225: frame1000.jpg
Working 6/2225: frame1001.jpg
Working 7/2225: frame1002.jpg
Working 8/2225: frame1003.jpg
Working 9/2225: frame1004.jpg
Working 10/2225: frame1005.jpg
Working 11/2225: frame1006.jpg
Working 12/2225: frame1007.jpg
Working 13/2225: frame1008.jpg
Working 14/2225: frame1009.jpg
Working 15/2225: frame101.jpg
Working 16/2225: frame1010.jpg
Working 17/2225: frame1011.jpg
Working 18/2225: frame1012.jpg
Working 19/2225: frame1013.jpg
Working 20/2225: frame1014.jpg
Working 21/2225: frame1015.jpg
Working 22/2225: frame1016.jpg
Working 23/2225: frame1017.jpg
Working 24/2225: frame1018.jpg
Working 25/2225: frame1019.jpg
Working 26/2225: frame102.jpg
Working 27/2225: frame1020.jpg
Working 28/2225: frame1021.jpg
Working 29/2225: frame1022.jpg
Working 

,Image number,Square,Radius,Circumference,Alcohol concentration
0,1000,2100278.52,817.64,5648.95,0.0
1,1001,2100244.83,817.64,5646.17,0.0
2,1002,2100626.67,817.71,5646.17,0.0
3,1003,2152624.60,827.77,5563.44,0.0
4,1014,2068900.07,811.51,5602.51,0.0
...,...,...,...,...,...
18881,995,493574.57,396.37,3402.35,75.0
18882,996,489021.78,394.54,2977.73,75.0
18883,997,488931.28,394.50,2972.45,75.0
18884,998,462435.95,383.66,3704.64,75.0


In [ ]:
image_paths_96 = glob.glob("/content/extracted_frames_96%/*.jpg")
image_paths_96.sort()
add_concentration_data(image_paths_96, 96, "bubble_analysis_um.csv")

Выходные данные были обрезаны до нескольких последних строк (5000).
Working 64/5063: frame1054.jpg
Working 65/5063: frame1055.jpg
Working 66/5063: frame1056.jpg
Working 67/5063: frame1057.jpg
Working 68/5063: frame1058.jpg
Working 69/5063: frame1059.jpg
Working 70/5063: frame106.jpg
Working 71/5063: frame1060.jpg
Working 72/5063: frame1061.jpg
Working 73/5063: frame1062.jpg
Working 74/5063: frame1063.jpg
Working 75/5063: frame1064.jpg
Working 76/5063: frame1065.jpg
Working 77/5063: frame1066.jpg
Working 78/5063: frame1067.jpg
Working 79/5063: frame1068.jpg
Working 80/5063: frame1069.jpg
Working 81/5063: frame107.jpg
Working 82/5063: frame1070.jpg
Working 83/5063: frame1071.jpg
Working 84/5063: frame1072.jpg
Working 85/5063: frame1073.jpg
Working 86/5063: frame1074.jpg
Working 87/5063: frame1075.jpg
Working 88/5063: frame1076.jpg
Working 89/5063: frame1077.jpg
Working 90/5063: frame1078.jpg
Working 91/5063: frame1079.jpg
Working 92/5063: frame108.jpg
Working 93/5063: frame1080.jpg
Worki

,Image number,Square,Radius,Circumference,Alcohol concentration
0,1000,2100278.52,817.64,5648.95,0.0
1,1001,2100244.83,817.64,5646.17,0.0
2,1002,2100626.67,817.71,5646.17,0.0
3,1003,2152624.60,827.77,5563.44,0.0
4,1014,2068900.07,811.51,5602.51,0.0
...,...,...,...,...,...
16656,995,287745.63,302.64,5433.36,96.0
16657,996,272354.43,294.44,5814.69,96.0
16658,997,248324.92,281.15,4795.49,96.0
16659,998,261184.72,288.34,5012.87,96.0
